In [3]:
from google.colab import files
uploaded = files.upload()


Saving sales_data_sample.csv to sales_data_sample.csv


In [30]:
import sqlite3
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')


# Loading CSV data into pandas DataFrame
df = pd.read_csv('sales_data_sample.csv', encoding='latin1')


In [31]:
# Preparing and cleaning data for SQLite insertion
# Renaming columns
df_sales = df[['PRODUCTCODE', 'QUANTITYORDERED', 'PRICEEACH']].copy()
df_sales.columns = ['product', 'quantity', 'price']
df_sales['quantity'] = df_sales['quantity'].astype(int)
df_sales['price'] = df_sales['price'].astype(float)

In [32]:
# Creating SQLite Database
conn = sqlite3.connect('sales_data.db')
cursor = conn.cursor()

In [33]:
cursor.execute('DROP TABLE IF EXISTS sales')

In [34]:
cursor.execute('''
CREATE TABLE sales (
    product TEXT,
    quantity INTEGER,
    price REAL
)
''')

In [35]:
# Insert data into sales table
df_sales.to_sql('sales', conn, if_exists='append', index=False)

2823

In [36]:
 ## Query
query = '''
SELECT product,
       SUM(quantity) AS total_qty,
       SUM(quantity * price) AS revenue
FROM sales
GROUP BY product
ORDER BY revenue DESC
'''

df_summary = pd.read_sql_query(query, conn)

In [37]:
# Summary results
print("Sales Summary by Product:")
print(df_summary)

Sales Summary by Product:
      product  total_qty    revenue
0    S18_3232       1774  176026.63
1    S24_3856       1052  103489.89
2    S18_4600       1031  101835.00
3    S24_2300        996   99600.00
4    S18_2238        966   96300.00
..        ...        ...        ...
104  S24_2022        851   42524.98
105  S24_2972        912   42145.94
106  S32_2206        836   39237.71
107  S24_1937        844   37392.38
108  S24_3969        745   33181.66

[109 rows x 3 columns]


In [38]:
# Plotting a bar chart of revenue by product (Top 10)
top_n = 10
plt.figure(figsize=(16, 8))
df_top = df_summary.head(top_n)
df_top.plot(kind='bar', x='product', y='revenue', legend=False)
plt.title('Top 10 Products by Revenue')
plt.xlabel('Product')
plt.ylabel('Revenue')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


In [39]:
plt.savefig('top_sales_revenue_chart.png')

In [19]:
plt.show()

In [20]:
conn.close()